In [2]:
# 필수 라이브러리 호출 
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 

from sklearn.metrics import accuracy_score as acc_sc

In [5]:
st_default_org = pd.read_csv('./st_default.csv')

In [10]:
st_default_df = st_default_org.iloc[:,1:]

In [11]:
# 전체 데이터
st_default_df 

# X_data와 y_target을 정의한다. 
X_data =st_default_df.drop('default', axis=1) # st_defafult_df.iloc[:,1:]
y_target = st_default_df.default

In [12]:
# EDA를 통한 데이터 확이 
X_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   student  10000 non-null  object 
 1   balance  10000 non-null  float64
 2   income   10000 non-null  float64
dtypes: float64(2), object(1)
memory usage: 234.5+ KB


In [15]:
X_data.describe()

,balance,income
count,10000.000000,10000.000000
mean,835.374886,33516.981876
std,483.714985,13336.639563
min,0.000000,771.967729
25%,481.731105,21340.462905
50%,823.636973,34552.644800
75%,1166.308387,43807.729275
max,2654.322576,73554.233500


In [17]:
# 크기 확인
print('X_data의 크기:', X_data.shape[0])
print('X_data의 타입:', type(X_data)) # 2차원

print('y_target의 크기:', y_target.shape[0])
print('y_target의 타입:', type(y_target)) # 1차원

X_data의 크기: 10000
X_data의 타입: <class 'pandas.core.frame.DataFrame'>
y_target의 크기: 10000
y_target의 타입: <class 'pandas.core.series.Series'>


In [20]:
# 지도학습 :: 분류 - 종속: 범주 -> label encoding vs label encoding
#            회귀 - 종속: 연속 

# 문자들을 숫자로 변환한다. 
## 1) 그냥 문자(학생) 컬럼을 없애면? 
## 2) 문자 -> one-hot // Label encoding

X_data_df = pd.get_dummies(X_data) # one-hot :: ndarray의 경우엔 문자 -> 레이블 -> 원핫 

In [21]:
y_target[:5] # series

0    No
1    No
2    No
3    No
4    No
Name: default, dtype: object

In [25]:
# y 값에는 Label encoding(레이블 인코딩)
from sklearn.preprocessing import LabelEncoder

le_encode = LabelEncoder()
y_target_scaled = le_encode.fit_transform(y_target)

# y_target값을 확인 - imbalanced data(쏠림 데이터)
pd.Series(y_target_scaled).value_counts()

0    9667
1     333
dtype: int64

In [28]:
# 데이터 분할 
X_train, X_test, y_train, y_test = train_test_split(X_data_df, y_target_scaled, stratify=y_target_scaled)

In [34]:
# 의사결정 나무 적용 
dt_clf = DecisionTreeClassifier()
dt_clf.fit(X_train, y_train) # 학습을 수행 

# 예측
y_pred = dt_clf.predict(X_test)

In [36]:
# 평가지표 
print(f'정확도는:{acc_sc(y_pred, y_test)}')

정확도는:0.9568


In [39]:
# CV - 교차검증을 수행 :: cross_val_score
from sklearn.model_selection import cross_val_score

# 전체 데이터 얼마나 CV의 정확도?
scores = cross_val_score(dt_clf, X_train, y_train, scoring='accuracy', cv=10)
print(f'교차 검증별 정확도:{np.round(scores, 4)}')
print(f'평균 검증별 정확도:{np.round(np.mean(scores), 4)}')

교차 검증별 정확도:[0.9507 0.9507 0.9573 0.9507 0.9547 0.9413 0.9453 0.9627 0.9627 0.9547]
평균 검증별 정확도:0.9531


In [40]:
y_target.value_counts()

No     9667
Yes     333
Name: default, dtype: int64

In [ ]:
# 1. 왜 정확도의 값이 높게 나왔을까?
# 2. 데이터는 어떠한 특징을 지니고 있을까?
# 3. 데이터에는 어떤 문제점이 있나?
# 4. 그렇다면 이러한 문제점을 해결하기 위한 방안은 무엇?

# [1] oversampling vs undersampling
# [2] Data augumentation - 이미지 분석에서 많이 쓰임
# [3] get more data
# [4] change our metrics (평가지료를 다른 것으로 사용 ex. precision, recall, f1-score, roc_auc )